## 生成特徵：STFT 每個頻率成分個各種統計量共 10 * 129 = 1290 個特徵

In [ ]:
import numpy as np 
import pandas as pd
import os
from tqdm import tqdm,tqdm_notebook
from scipy.signal import stft
import matplotlib.pyplot as plt

In [ ]:
# Import
float_data = pd.read_csv("../input/train.csv", dtype={"acoustic_data": np.float32, "time_to_failure": np.float32}).values

In [ ]:
from sklearn.linear_model import LinearRegression

def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]
def add_trend_feature_abs(arr, abs_values=True):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

In [ ]:
def create_feature(arr):
    fft_spe = abs(stft(wave)[2])[:,:]
    q3=np.quantile(fft_spe,0.75,axis=1)
    q2=np.quantile(fft_spe,0.5,axis=1)
    q1=np.quantile(fft_spe,0.25,axis=1)
    trend = np.apply_along_axis(add_trend_feature,1,fft_spe)
    trend_abs = np.apply_along_axis(add_trend_feature_abs,1,fft_spe)
    return np.r_[fft_spe.mean(axis=1),fft_spe.std(axis=1),fft_spe.max(axis=1),fft_spe.min(axis=1),
                 q2,q3,q1,q3-q1,trend,trend_abs]

In [ ]:
segment = 150000
total = int(len(float_data)/150000)
f_list = []
target = []
for i in tqdm_notebook(range(total)):
    wave = float_data[i*segment:(i+1)*segment,0]
    f_list.append(create_feature(wave))
    target.append(float_data[i*segment+149999,1])

In [ ]:
col = []
for sta in ['Mean','Std','Max','Min','Q2','Q3','Q1','IQR','Trend','Trend_abs']:
    for i in range(129):
        col += [f'fft_{i}_{sta}']
    

In [ ]:
X = pd.DataFrame(np.array(f_list),columns=col)
y = pd.Series(target)

In [ ]:
submit = pd.read_csv('../input/sample_submission.csv')

In [ ]:
f_list = []
for seg_id in tqdm_notebook(submit.seg_id):
    wave = pd.read_csv(f'../input/test/{seg_id}.csv').values[:,0]
    f_list.append(create_feature(wave))

In [ ]:
X_test = pd.DataFrame(f_list,index=submit.seg_id,columns=col)

In [ ]:
X.to_csv('X.csv')
y.to_csv('y.csv')
X_test.to_csv('X_test.csv')